# Árboles de decisión

Esta semana estudiaremos una familia de métodos muy populares y poderosos para clasificación y regresión: los modelos basados en árboles.

La estructura que utilizan estos métodos es el conocido **arbol de decisión**. Un árbol de decisión es un **árbol binario** en el que, en cada nodo del árbol, se evalúa un **predicado lógico** sobre el dato en cuestión; es decir, se hace una pregunta cuya respuesta es ``TRUE`` o ``FALSE``. Dependiendo de la respuesta, decidimos cuál rama del árbol tomar, hasta llegar a las hojas del árbol. **En las hojas del árbol se encuentra nuestro objetivo de clasificación final**.

Retomemos el ejemplo de los exámenes. Supongamos que hemos entrenado correctamente un modelo para clasificar, obteniendo el siguiente arbol:

<img src="img/arbol.png" width=600>

A la hora de clasificar un objeto, el "proceso" sería el siguiente:

- ¿Estudió más de tres horas?
    - NO: **``REPROBADO``**
    - SI: ¿Durmió más de cuatro horas?
        - NO: **``REPROBADO``**
        - SI: **``APROBADO``**
        
## Inferencia

Hasta ahora todo muy bien, el problema es **cómo construir un árbol de decisión dado un dataset**. Los algoritmos "convencionales", como **CART**, **ID3** y **C4.5**, funcionan de manera **recursiva** y siguen el siguiente esquema:

1. Enumeramos un conjunto de **predicados** sobre los atributos que tenemos, sobre distintos valores posibles
    - Por ejemplo: 
        - ``(horas_estudio > 0)`` 
        - ``(horas_estudio > 0.5)``
        - ``(horas_estudio > 1)``, ...,
        - ``(horas_sueno > 0)``, 
        - ``(horas_sueno > 1)``, ...
        - ``(horas_sueno > 2)``, ...
        
        
2. Con cada uno de estos predicados, evaluamos su **ganancia de información**.
    - Esta es una medida de que tan bien un predicado "parte" el dataset.
        - Por ejemplo, ``horas_estudio > 0`` no lo parte muy bien, puesto que quedamos con la misma mezcla de ``APROBADO`` y ``REPROBADO``.
        - Pero ``horas_estudio > 3`` divide mejor: los que no cumplen con este predicado son, en su mayoría, ``REPROBADO``.
        
    - Más adelante veremos varias medidas de ganancia de información.
    
    
3. De todos los predicados, escogemos el que tiene la **mejor ganancia de información**.
    - Por ejemplo, ``horas_estudio > 3``.
    
    
4. Dividemos el dataset en dos particiones: **quienes cumplen** con el predicado que escogimos **y quienes no cumplen**. 
    - Es decir, ``P1``: quienes estudiaron más de tres horas y ``P2``: quienes no lo hicieron.
    
    

5. Repetimos este proceso recursivamente (desde el paso 1) con ambas particiones del dataset y "unimos" los subárboles resultantes.

6. El proceso se detiene para una partición dada cuando:
    - En dicha partición todos los elementos tienen la misma categoría
    - Ninguno de los predicados provee ganancia de información
    - Otras restricciones basadas en constantes: profundidad máxima del árbol alcanzada o tamaño mínimo de la partición no se cumple.
    
    - **Al detenernos**, la partición se convertirá en una hoja del árbol y se le asignará la clasificación a la categoría mayoritaria.
    
### Medidas de ganancia de información

A la hora de evaluar cuál predicado particiona mejor el dataset, evaluamos con cada uno una medida de **ganancia de información**. Esta es una medición que se hace sobre la **distribución resultante al aplicar el predicado al dataset**.

Volvamos a nuestro dataset del examen y veamos cómo se distribuyen los resultados:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

horas_estudio = np.array([5, 4, 1, 1, 1, 2, 3, 3, 2, 0, 3, 3, 2, 3, 2, 3, 1, 5, 4, 4, 3, 4, 2, 4, 3, 3, 6, 4, 4, 5, 3, 4, 3, 2, 6, 2, 4, 4, 1, 5, 3, 3, 5, 7, 4, 3, 5, 2, 3, 5, 3, 4, 3, 3, 1, 3, 6, 0, 4, 3, 7, 2, 5, 3, 3, 4, 3, 1, 5, 4, 1, 4, 3, 1, 2, 4, 2, 2, 2, 2, 4, 3, 3, 1, 4, 2, 1, 0, 4, 4, 3, 3, 4, 6, 5, 3, 2, 2, 6, 6])
horas_sueno = np.array([4, 4, 5, 5, 4, 6, 7, 7, 5, 8, 5, 7, 7, 7, 7, 7, 4, 8, 4, 8, 6, 8, 5, 4, 5, 5, 7, 4, 6, 8, 6, 4, 7, 4, 5, 7, 7, 4, 6, 6, 5, 6, 7, 6, 4, 8, 7, 4, 4, 6, 5, 8, 6, 8, 4, 5, 4, 6, 8, 5, 6, 6, 5, 6, 4, 8, 4, 6, 5, 7, 5, 6, 7, 7, 8, 4, 4, 6, 8, 6, 4, 6, 5, 5, 5, 4, 4, 7, 8, 5, 4, 8, 5, 6, 5, 6, 7, 7, 4, 5])
resultado_examen = np.array(['APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO'])

plt.hist(resultado_examen)
plt.show()

print('Porcentaje APROBADO: ' + str(100*(resultado_examen == 'APROBADO').mean()))
print('Porcentaje REPROBADO: ' + str(100*(resultado_examen == 'REPROBADO').mean()))


Ahora vamos a introducir las dos medidas de ganancia de información más populares **para clasificación**: la **entropia de información** y el **coeficiente de Gini**. Ambas funciones se calculan sobre una distribución discreta de valores o histograma $D$. 

La entropía de información se define como $$\mathrm{Ent}\left(D\right) = -\sum_{d \in D} p_d \mathrm{log} \left(p_d\right)$$

Mientras que el coeficiente de Gini es  $$\mathrm{Gini}\left(D\right) = 1 - \sum_{d \in D} \left(p_d\right)^2$$

**Por ejemplo**, en la distribución anterior, los valores de estas medidas son

$$\mathrm{Ent}\left(D\right) = - \left(0.63 \times \mathrm{log}\left(0.63 \right) + 0.37 \times \mathrm{log}\left(0.37 \right)\right) = 0.6589557$$

$$\mathrm{Gini}\left(D\right) = 1 - \left(0.63^2 + 0.37^2 \right) = 0.4662$$

Véamos cómo se comportan estas funciones con distintas distribuciones de valores




In [ ]:
def entropia(D):
    return -(D*np.log(D)).sum(axis=-1)

def gini(D):
    return 1 - (D**2).sum(axis=-1)

## Generamos distintos valores de prueba entre 0 y 1
x = np.linspace(0.0001,0.9999,100)
x = np.array((x,1-x)).T

plt.plot(x, entropia(x))
plt.title("Entropia")
plt.show()
plt.plot(x, gini(x))
plt.title("Gini")
plt.show()

Como podemos ver, ambas medidas se parecen mucho y alcanzan su **mayor valor en 0.5**. Esto lo que quiere decir es que una distribución uniforme $\left(0.5, 0.5\right)$ es la que lleva **mayor incerditumbre**. El objetivo de la inferencia, entonces, es **buscar predicados que creen particiones de baja entropía o bajo Gini**, es decir, **las particiones más desiguales posibles**.

Sin embargo, a la hora de evaluar un predicado, lo debemos hacer con las **dos distribuciones resultantes**: la que resulta cuando el predicado es verdadero, y la que resulta cuando el predicado es falso. La **ganancia total**, entonces, es la **media ponderada de ambas ganancias**:

$$-G = \frac{n_{T}}{N}G_T + \frac{n_F}{N}G_F$$

donde $N$ es el tamaño del dataset antes de ser particionado, $n_T$ es el tamaño de la partición que evalúa a **verdadero** mientras que $n_F$ es el tamaño de la partición que evalúa a **falso**, y $G_T$ y $G_F$ las ganancias de las respectivas particiones.

Véamos como ejemplo el predicado ``horas_estudio < 3`` utilizando Gini y dibujamos los histogramas correspondientes:

In [ ]:

## Tamaño dataset
N = resultado_examen.shape[0]

## Partición donde horas_estudio < 3 (verdadera)
d_T = resultado_examen[horas_estudio < 3]
## Partición donde horas_estudio >= 3 (falsa)
d_F = resultado_examen[horas_estudio >= 3]

plt.hist(resultado_examen)
plt.title('Distribucion original')
plt.show()

plt.hist(d_T)
plt.title('Distribucion de particion horas_estudio < 3')
plt.show()

plt.hist(d_F)
plt.title('Distribucion de particion horas_estudio >= 3')
plt.show()


## Tamaños de ambas particiones
n_T = d_T.shape[0]
n_F = d_F.shape[0]

vT = (d_T == 'APROBADO').mean()
vF = (d_F == 'REPROBADO').mean()

## Ginis de ambas particiones
gini_T = gini(np.array((vT, 1-vT)))
gini_F = gini(np.array((vF, 1-vF)))

## Ganancia total
G = (n_T * gini_T)/N + (n_F * gini_F)/N

print('N =', N)
print('n_T =', n_T)
print('n_F =', n_F)
print('gini_T =', gini_T)
print('gini_F =', gini_F)
print('G =', G)

Como podemos ver, el predicado ``horas_estudio < 3`` es **excelente** ya que divide el dataset en dos particiones muy desiguales: 1) una donde la mayoría son ``APROBADO`` y 2) otra donde la mayoría son ``REPROBADO``.

De hecho, confirmemos esto probando con los valores del 1 al 5 en distinos predicados y viendo su ganancia total con Gini:

In [ ]:
def distrib_pred(he_x):

    d_T = resultado_examen[horas_estudio < he_x]
    d_F = resultado_examen[horas_estudio >= he_x]
    n_T = d_T.shape[0]
    n_F = d_F.shape[0]
    
    vT = (d_T == 'APROBADO').mean()
    vF = (d_F == 'REPROBADO').mean()
    gini_T = gini(np.array((vT, 1-vT)))
    gini_F = gini(np.array((vF, 1-vF)))

    return (n_T * gini_T)/N + (n_F * gini_F)/N

lp = np.linspace(1, max(horas_estudio), max(horas_estudio))
vals = pd.DataFrame([ (x, distrib_pred(x)) for x in lp ], columns=['x', 'Gain'])
vals

Queda demostrado que el valor de ``x`` de 3.0 es el que reduce el valor y por ende el mejor predicado para dividir el dataset. 

## Haciendo regresión

Es posible utilizar árboles de decisión para hacer regresión. Pero debemos resolver dos problemas:

1. En las hojas del árbol, la asignación de la clasificación se hace a la **clase mayoritaria**, pero en regresión nos topamos con un conjunto de números reales que **pueden ser todos distintos**. 

2. Las medidas de ganancia como el coeficiente de Gini o la entropía de la información son para distribuciones **discretas**, pero ahora tenemos que trabajar con una distribución **contínua** de valores numéricos.


El problema (1) se resuelve utilizando la **media de la partición correspondiente a la hoja**. También se puede usar la mediana o el punto medio entre el valor máximo y el mínimo en la partición.

El problema (2) se resuelve utilizando como ganancia **la varianza de cada partición resultante de un predicado**. De modo que buscamos **predicados que reducen la varianza**. 